In [1]:
from PIL import Image as PilImage
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import Image, display
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras import metrics
from sklearn.metrics import precision_score
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# def resize_image(input_image_path, output_image_path, size):
#     original_image = Image.open(input_image_path)
#     width, height = original_image.size
#     print(f"The original image size is {width} wide x {height} tall")

#     resized_image = original_image.resize(size)
#     width, height = resized_image.size
#     print(f"The resized image size is {width} wide x {height} tall")
#     resized_image.show()
#     resized_image.save(output_image_path)

In [3]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Coachwhip.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Coachwhip_resized.jpg', (950, 660))

In [4]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Pinesnake.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Eastern Pinesnake_resized.jpg', (950, 660))

In [5]:
# resize_image('C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Short-tailed Kingsnake.jpg', 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos\\Short-tailed Kingsnake_resized.jpg', (950, 660))

In [6]:
image_dir = 'C:\\Users\\howar\\Desktop\\Capstone\\FloridaSnakePredictor\\Snake Photos'

# # Get list of image file names
# image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# # Create DataFrame
# df = pd.DataFrame(image_files, columns=['Image'])

# # Function to display image
# def display_image(filename):
#     return display(Image(filename=os.path.join(image_dir, filename)))

# # Apply function to DataFrame
# df['Image'].apply(display_image)

In [7]:
all_images = set(f for f in os.listdir(image_dir) if f.endswith('.jpg'))

In [8]:
venomous_snakes = ["Florida Cottonmouth.jpg", "Dusky Pygmy Rattlesnake.jpg", "Eastern Diamond-backed Rattlesnake.jpg", "Harlequin Coralsnake.jpg", "Eastern Copperhead.jpg", "Timber Rattlesnake.jpg"]
venomous_snakes = set(venomous_snakes)
non_venomous_snakes = all_images - venomous_snakes

In [9]:
venomous_snakes = list(venomous_snakes)
non_venomous_snakes = list(non_venomous_snakes)



df = pd.DataFrame({
    'Venomous': pd.Series(venomous_snakes),
    'Non-venomous': pd.Series(non_venomous_snakes)
})

print(df)

                                  Venomous                        Non-venomous
0                   Eastern Copperhead.jpg              Midland Watersnake.jpg
1                  Florida Cottonmouth.jpg                   Red Cornsnake.jpg
2                 Harlequin Coralsnake.jpg             African Rock Python.jpg
3   Eastern Diamond-backed Rattlesnake.jpg                Brown Watersnake.jpg
4                   Timber Rattlesnake.jpg        Plain-bellied Watersnake.jpg
5              Dusky Pygmy Rattlesnake.jpg            Red-bellied Mudsnake.jpg
6                                      NaN                Rough Greensnake.jpg
7                                      NaN       Eastern Coachwhip_resized.jpg
8                                      NaN      Southeastern Crowned Snake.jpg
9                                      NaN        Southern Hog-nosed Snake.jpg
10                                     NaN             Eastern Ribbonsnake.jpg
11                                     NaN          

In [10]:
venomous_snakes

['Eastern Copperhead.jpg',
 'Florida Cottonmouth.jpg',
 'Harlequin Coralsnake.jpg',
 'Eastern Diamond-backed Rattlesnake.jpg',
 'Timber Rattlesnake.jpg',
 'Dusky Pygmy Rattlesnake.jpg']

In [11]:
df

,Venomous,Non-venomous
0,Eastern Copperhead.jpg,Midland Watersnake.jpg
1,Florida Cottonmouth.jpg,Red Cornsnake.jpg
2,Harlequin Coralsnake.jpg,African Rock Python.jpg
3,Eastern Diamond-backed Rattlesnake.jpg,Brown Watersnake.jpg
4,Timber Rattlesnake.jpg,Plain-bellied Watersnake.jpg
5,Dusky Pygmy Rattlesnake.jpg,Red-bellied Mudsnake.jpg
6,NaN,Rough Greensnake.jpg
7,NaN,Eastern Coachwhip_resized.jpg
8,NaN,Southeastern Crowned Snake.jpg
9,NaN,Southern Hog-nosed Snake.jpg


In [12]:
# Get a list of all files in that directory
all_files = os.listdir(image_dir)

# Filter the list for files ending in '.jpg'
image_paths = [os.path.join(image_dir, file) for file in all_files if file.endswith('.jpg')]

In [13]:
# Initialize an empty list to store the numpy arrays
images = []

for path in image_paths:
    # Open the image file
    with PilImage.open(path) as img:
        # Convert the image data to a numpy array
        img_array = np.array(img)
        
        # Append the image array to the list
        images.append(img_array)

# Convert the list of image arrays into a single numpy array
images = np.array(images)

print(images.shape)
# Images, Height, Width, Color Channels

(55, 660, 950, 3)


In [14]:
labels = []

for path in image_paths:
    # Get the name of the snake from the path
    snake_name = os.path.basename(path)  # this gets the filename with the extension
    
    # Check if the snake is venomous
    if snake_name in venomous_snakes:
        # If it is, append 1 to the labels list
        labels.append(1)
    else:
        # If it's not, append 0 to the labels list
        labels.append(0)

# Convert the list of labels into a numpy array
labels = np.array(labels)
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [15]:


# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [16]:


# # Create the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(660, 950, 3)))  # change input shape to match your images
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))  # 1 because we have 2 classes


In [17]:
# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=[metrics.Precision()])

In [18]:
# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# # Early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# # Calculate class weights
# class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

# # Convert class weights to a dictionary to pass it to the fit function
# class_weight_dict = dict(enumerate(class_weights))

# # Define the undersampling method
# undersample = RandomUnderSampler(sampling_strategy='majority')

# # Flatten the image data
# X_train_flat = X_train.reshape(X_train.shape[0], -1)

# # Apply the undersampling method
# X_under_flat, y_under = undersample.fit_resample(X_train_flat, y_train)

# # Reshape the undersampled data back to its original shape
# X_under = X_under_flat.reshape(-1, 660, 950, 3)

# # Fit the model using the undersampled data
# model.fit(datagen.flow(X_under, y_under, batch_size=16),
#           epochs=20,
#           validation_data=(X_test, y_test),
#           callbacks=[early_stopping],
#           class_weight=class_weight_dict)


In [19]:
# Split your data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Calculate class weights
weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = dict(enumerate(weights))


# Define your model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(660, 950, 3)))  # change input shape to match your images
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # binary classification

# Compile your model
model.compile(optimizer='adam', loss='binary_crossentropy')

#Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train your model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=16, callbacks=[early_stopping])


C:\Users\howar\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=[0 1], y=[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/20
3/3 [==============================] - 4s 1s/step - loss: 17419.5273 - val_loss: 11876.5352
Epoch 2/20
3/3 [==============================] - 4s 1s/step - loss: 10938.6025 - val_loss: 8182.9292
Epoch 3/20
3/3 [==============================] - 4s 1s/step - loss: 7648.2671 - val_loss: 2825.4126
Epoch 4/20
3/3 [==============================] - 4s 1s/step - loss: 3938.8484 - val_loss: 536.6265
Epoch 5/20
3/3 [==============================] - 4s 1s/step - loss: 984.6438 - val_loss: 31.2644
Epoch 6/20
3/3 [==============================] - 4s 1s/step - loss: 165.1512 - val_loss: 4.0676
Epoch 7/20
3/3 [==============================] - 5s 2s/step - loss: 125.6664 - val_loss: 1.4246
Epoch 8/20
3/3 [==============================] - 4s 1s/step - loss: 15.6345 - val_loss: 1.6676
Epoch 9/20
3/3 [==============================] - 4s 1s/step - loss: 8.0093 - val_loss: 0.7479
Epoch 10/20
3/3 [==============================] - 5s 2s/step - loss: 0.2201 - val_loss: 0.5823
Epoch 11/20
3/3

In [20]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)

# print(f'Test loss: {loss:.4f}, Test precision: {precision*100:.2f}%')

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)  # round the predictions to 0 or 1

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # adjust average as needed

print(f'Test loss: {loss:.4f}, Test F1 score: {f1*100:.2f}%')


Test loss: 0.6402, Test F1 score: 68.90%
